In [7]:
import os
import numpy as np

def load_spectrograms(count):
    # Get the current working directory
    current_dir = os.getcwd()

    # Define the root paths for clean and noisy spectrograms using absolute paths
    clean_root = os.path.abspath(os.path.join(current_dir, f'../../dataset/iteration-{count}/data/clean/spectrogram'))
    noisy_root = os.path.abspath(os.path.join(current_dir, f'../../dataset/iteration-{count}/data/mixed/spectrogram'))

    # Dictionary to hold associated data
    data_pairs = []

    # Walk through the directory structure
    for subdir, dirs, files in os.walk(clean_root):
        for file in files:
            if file.endswith('.npy'):
                # Generate the full path to the clean file
                clean_path = os.path.join(subdir, file)

                # Corresponding noisy file path
                # Replace the root directory and append '_mixed' before the extension
                noisy_path = clean_path.replace(clean_root, noisy_root).rsplit('.', 1)[0] + '_mixed.npy'

                # Check if the noisy file exists
                if os.path.exists(noisy_path):
                    # Load the spectrograms
                    clean_spectrogram = np.load(clean_path)
                    noisy_spectrogram = np.load(noisy_path)

                    # Append the pair (noisy, clean) to the data list
                    data_pairs.append((noisy_spectrogram, clean_spectrogram))

    return data_pairs

# Example of how to set the 'count' variable and load data
count = 1  # User can change this as needed
data_pairs = load_spectrograms(count)

# Example of how to use the data
# Let's say you want to check the number of loaded pairs and print the shape of the first pair
print(f"Number of loaded data pairs: {len(data_pairs)}")
if data_pairs:
    print(f"Shape of first noisy spectrogram: {data_pairs[0][0].shape}")
    print(f"Shape of first clean spectrogram: {data_pairs[0][1].shape}")


Number of loaded data pairs: 0
